attentionの可視化をするコードを追記する予定です

In [1]:
target_model_dir_ = 'runs/Jun03_14-52-17_99986f60c588'

In [2]:
from pathlib import Path

target_model_dir = Path('..', target_model_dir_)
target_model_dir = target_model_dir.resolve()
assert target_model_dir.is_dir(), target_model_dir

In [3]:
import sys

path = str(target_model_dir / 'src')
if path not in sys.path:
    sys.path.insert(0, path)

del sys

In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torchvision.transforms.functional import to_pil_image, to_tensor
from torchvision.utils import make_grid
from torchinfo import summary

from IPython.display import display

In [5]:
from model.model import TransformerModel

model = TransformerModel()
model.eval()

summary(model, (2, *model.input_shape), device='cpu')

Layer (type:depth-idx)                        Output Shape              Param #
TransformerModel                              --                        --
├─Linear: 1-1                                 [2, 16, 64]               3,200
├─TransformerEncoder: 1-2                     [2, 16, 64]               --
│    └─ModuleList: 2                          --                        --
│    │    └─TransformerEncoderLayer: 3-1      [2, 16, 64]               25,216
│    │    └─TransformerEncoderLayer: 3-2      [2, 16, 64]               25,216
│    │    └─TransformerEncoderLayer: 3-3      [2, 16, 64]               25,216
├─Linear: 1-3                                 [2, 10]                   650
Total params: 79,498
Trainable params: 79,498
Non-trainable params: 0
Total mult-adds (M): 0.06
Input size (MB): 0.01
Forward/backward pass size (MB): 0.21
Params size (MB): 0.32
Estimated Total Size (MB): 0.54

In [6]:
weight_path = sorted(target_model_dir.glob('*.pt'))[-1]
print(weight_path)

model.load_state_dict(torch.load(weight_path))

/workspace/runs/Jun03_14-52-17_99986f60c588/epoch00029.pt


<All keys matched successfully>

In [7]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = MNIST(root='../data', train=False, download=False, transform=transform)

In [8]:
for index in range(20):
    print(index)
    image, label = dataset[index]

    display(to_pil_image(image))

    patches = model.unfold_patch(image[None])[0]
    display(to_pil_image(make_grid(patches, nrow=model.num_patches, pad_value=0.5)))

    output_prob = model(image[None])[0].softmax(0)

    print(output_prob.tolist())

    print(output_prob.argmax())

    print()

0


[0.004493934102356434, 0.05076932907104492, 0.018564721569418907, 0.002767853671684861, 0.01326053962111473, 0.014602843672037125, 0.00039116654079407454, 0.799828827381134, 0.0628809854388237, 0.03243980184197426]
tensor(7)

1


[0.0003057601279579103, 0.004064821172505617, 0.3416615426540375, 0.03231873735785484, 0.00042588802170939744, 0.503757119178772, 0.0001153868215624243, 0.11243999749422073, 0.0040403869934380054, 0.0008704166393727064]
tensor(5)

2


[0.0010110553121194243, 0.8871234059333801, 0.01853550784289837, 0.0001016608948702924, 0.0035742921754717827, 0.00030907473410479724, 0.0003630402497947216, 0.08735601603984833, 0.00012114025594200939, 0.001504821004346013]
tensor(1)

3


[0.008517214097082615, 0.39745986461639404, 0.017152957618236542, 0.000839269021525979, 0.1997540444135666, 0.009455758146941662, 0.05477270856499672, 0.24950018525123596, 0.02526239864528179, 0.03728560730814934]
tensor(1)

4


...

[2.9658849598490633e-05, 5.178767423785757e-06, 0.026920268312096596, 0.10358455032110214, 0.004209334962069988, 0.7330115437507629, 1.5115139831323177e-05, 0.00010911698336713016, 0.13097715377807617, 0.001138105639256537]
tensor(5)

16


[0.007004816085100174, 0.0029187402687966824, 0.3005478084087372, 0.0024778989609330893, 0.0014082287671044469, 0.0005992591031827033, 0.00013892119750380516, 0.45871683955192566, 0.026977380737662315, 0.1992100477218628]
tensor(7)

17


[0.0012657791376113892, 0.28657016158103943, 0.030393552035093307, 0.013096828944981098, 0.0414876788854599, 0.011506429873406887, 0.00014384313544724137, 0.5307593941688538, 0.01903774030506611, 0.06573858112096786]
tensor(7)

18


[0.00010903667862294242, 0.0005573397502303123, 0.5415269136428833, 0.015097130089998245, 5.480544132296927e-05, 0.43215277791023254, 0.0002343173837289214, 0.0076516796834766865, 0.002600670326501131, 1.534260081825778e-05]
tensor(2)

19


[0.0017095168586820364, 0.010257943533360958, 0.02934359572827816, 0.0004997751675546169, 0.10316552966833115, 0.00016673619393259287, 7.55499058868736e-07, 0.7894846200942993, 0.0014954055659472942, 0.06387613713741302]
tensor(7)



In [9]:
x = image[None]
x = model.unfold_patch(x)
x = x.flatten(2)
x = model.patch_embedding(x)
x = model.transformer_encoder(x)
x = x.mean(1)
x = model.mlp_head(x)
x.shape, x[0].softmax(0).tolist()

(torch.Size([1, 10]),
 [0.0017095168586820364,
  0.010257943533360958,
  0.02934359572827816,
  0.0004997751675546169,
  0.10316552966833115,
  0.00016673619393259287,
  7.55499058868736e-07,
  0.7894846200942993,
  0.0014954055659472942,
  0.06387613713741302])

In [10]:
list(model.transformer_encoder.children())[0]

ModuleList(
  (0): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.5, inplace=False)
    (dropout2): Dropout(p=0.5, inplace=False)
  )
  (1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
    )
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((64,), eps=

In [11]:
model

TransformerModel(
  (patch_embedding): Linear(in_features=49, out_features=64, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.5, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.5, inplace=False)
        (dropout2): Dropout(p=0.5, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=